Convert sac scalar field to vtk 3d format

This current writer works for scalar variables
there is a problem with the input file because row zero appears to have co-ordinate values and not the required
density values for perturbed density

In [1]:
import numpy as np
from numpy import *
import scipy.io
from scipy import special
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import struct

In [28]:
inputfile='/media/mike/data/mike/proj/smaug/hyp/zeroHYP_265000.out'

#file = open('/media/mike/data/mike/proj/smaug/hyp/zeroHYP_265000.out','rb')


#create an output folder called vtk
outfilename='nypnov_ascii_265000.vtk'
outputfile='/media/mike/data/mike/proj/smaug/hyp/vtk/'+outfilename

#typedef enum vars {rho, mom1, mom2, energy, b1, b2,energyb,rhob,b1b,b2b} CEV;

#density scalar component
field=0
outfieldname='dens'

#origin
xo=0.0
yo=0.0
zo=0.0

#spacing
dx=1.90469e17
dy=1.90469e17
dz=1.90469e17

read sac file

In [3]:
file = open(inputfile,'rb')

file.seek(0,2)
eof = file.tell()
file.seek(0,0)

name = file.read(79)
nit = fromfile(file,dtype=int32,count=1)
t = fromfile(file,dtype=float64,count=1)
ndim=fromfile(file,dtype=int32,count=1)
neqpar=fromfile(file,dtype=int32,count=1)
nw=fromfile(file,dtype=int32,count=1)
#ndata = fromfile(file,dtype=int32,count=ndim)[:ndim]
ndata = fromfile(file,dtype=int32,count=3)
varbuf = fromfile(file,dtype=float,count=6)[:6]
varnames = file.read(79)

#typedef enum vars {rho, mom1, mom2, energy, b1, b2,energyb,rhob,b1b,b2b} CEV;
datcount=(nw+ndim)*ndata[0]*ndata[1]*ndata[2]

#alldat=fromfile(file,dtype=float,count=datcount)[:(nw+ndim)*ndata[0]*ndata[1]]
alldat=fromfile(file,dtype=float,count=datcount[0])
if size(alldat)<(nw+ndim)*ndata[0]*ndata[1]*ndata[2]:
    alldat=resize(alldat,(nw+ndim)*ndata[0]*ndata[1]*ndata[2])
alldat=np.reshape(alldat,(nw[0]+ndim[0],ndata[2],ndata[1],ndata[0]),'C') #opposite way to what you think!
#alldat=np.reshape(alldat,(12,256,256),'C')

file.close()

In [29]:
print(nw)
print(ndim)
print(ndata)

print(alldat[1,0,:,0])

[13]
[3]
[128 128 128]
[ 1.27798387e+19 -1.21900000e+19 -1.19933871e+19 -1.17967742e+19
 -1.16001613e+19 -1.14035484e+19 -1.12069355e+19 -1.10103226e+19
 -1.08137097e+19 -1.06170968e+19 -1.04204839e+19 -1.02238710e+19
 -1.00272581e+19 -9.83064516e+18 -9.63403226e+18 -9.43741935e+18
 -9.24080645e+18 -9.04419355e+18 -8.84758065e+18 -8.65096774e+18
 -8.45435484e+18 -8.25774194e+18 -8.06112903e+18 -7.86451613e+18
 -7.66790323e+18 -7.47129032e+18 -7.27467742e+18 -7.07806452e+18
 -6.88145161e+18 -6.68483871e+18 -6.48822581e+18 -6.29161290e+18
 -6.09500000e+18 -5.89838710e+18 -5.70177419e+18 -5.50516129e+18
 -5.30854839e+18 -5.11193548e+18 -4.91532258e+18 -4.71870968e+18
 -4.52209677e+18 -4.32548387e+18 -4.12887097e+18 -3.93225806e+18
 -3.73564516e+18 -3.53903226e+18 -3.34241935e+18 -3.14580645e+18
 -2.94919355e+18 -2.75258065e+18 -2.55596774e+18 -2.35935484e+18
 -2.16274194e+18 -1.96612903e+18 -1.76951613e+18 -1.57290323e+18
 -1.37629032e+18 -1.17967742e+18 -9.83064516e+17 -7.86451613e+17
 -

write vtk

In [33]:



# alldat :: the full array of vacdata from getpict 
# alldat[0:2] :: are the position data     
# field :: which field
#         e.g. 1,2,3
# vecsize :: 1,2,3 how many components field has
#        ; e.g. magnetic field, velocity or momentum
# outputfile :: is a string of the name of the output file (without
#; .vtk)

#all data is contained in an array alldat of shape nfields+ndim,dim[0],dim[1]

#start writing vtk file

byteorder='LittleEndian'
dim=ndata

#Define grid data size
sizew=dim


# Open the file .vtr
file = open(outputfile,'w')
 
# Header

#     printf,lu,'# vtk DataFile Version 2.0'
file.write("# vtk DataFile Version 2.0\n")

#     printf,lu,'Structured Grid'
#     printf,lu,'ASCII'
#     printf,lu,' '
#     printf,lu,'DATASET RECTILINEAR_GRID'
#     printf,lu,'DIMENSIONS ',sizew(1),' ',sizew(2),'    ',sizew(3)
file.write("Structured Grid\n")
file.write("ASCII\n")
file.write(" \n")
file.write("DATASET STRUCTURED_POINTS\n")
file.write("DIMENSIONS "+str(dim[0])+" "+str(dim[1])+" "+str(dim[2])+"\n")
file.write("SPACING "+str(dx)+" "+str(dy)+" "+str(dz)+"\n")
file.write("ORIGIN "+str(xo)+" "+str(yo)+" "+str(zo)+"\n")

#        printf,lu,'X_COORDINATES ',sizew(1),' double'
#        for ix=0,sizew(1)-1 do begin
#           printf,lu,x(ix,0,0)
#        endfor

##file.write("X_COORDINATES "+str(dim[0])+" double\n")
##j=0
##line=""
##for i1 in range(dim[0]):
##    line=line+str(alldat[1,0,i1,0])
##    line=line+"\n"
##file.write(line)

##file.write("Y_COORDINATES "+str(dim[1])+" double\n")
##j=1
##line=""
##for i1 in range(dim[1]):
##    line=line+str(alldat[1,0,i1,0])
##    line=line+"\n"
##file.write(line)

##file.write("Z_COORDINATES "+str(dim[2])+" double\n")
##j=2
##line=""
##for i1 in range(dim[2]):
##    line=line+str(alldat[1,0,i1,0])
##    line=line+"\n"
##file.write(line)

#        printf,lu,'POINT_DATA ',sizew(1)*sizew(2)*sizew(3)
#       printf,lu,'SCALARS ',filename,' double 1'
file.write("POINT_DATA "+str(dim[0]*dim[1]*dim[2])+"\n")
file.write("SCALARS "+outfieldname+" double 1\n")

file.write("LOOKUP_TABLE default\n")
#        printf,lu,'LOOKUP_TABLE TableName '
#        for iz=0,sizew(3)-1 do begin
#           for iy=0,sizew(2)-1 do begin
#              for ix=0,sizew(1)-1 do begin
#                printf,lu,vacdata(ix,iy,iz,field)
#              endfor
#           endfor
#        endfor
j=field+ndim
for i3 in range(dim[2]):
    for i2 in range(dim[1]):
        line=""
        for i1 in range(dim[0]):
#TODO
# note form in which we have written this otherwise 
#fields formatted with [] around numbers
            for d in alldat[j,i1,i2,i3]:
                line=line+str(d)+" "
        line=line+"\n"
        file.write(line)

file.close()

In [23]:
print(shape(alldat))

(16, 128, 128, 128)


In [25]:
print(128*128*128)
print(128*128)

2097152
16384
